In [ ]:
import os
import pandas as pd 
import numpy as np
import multiprocessing 
from multiprocessing import Pool 
from functools import partial 
from data_loader import data_loader_v2 

from sklearn.ensemble import RandomForestClassifier
import joblib 
from datetime import datetime

import lightgbm as lgb


In [ ]:
train_folder = 'data/train/'
test_folder = 'data/test/'
train_label_path = 'data/train_label.csv'

In [ ]:
train_list = os.listdir(train_folder)
test_list = os.listdir(test_folder)
train_label = pd.read_csv(train_label_path, index_col=0)

In [ ]:
num_class = len(train_label['label'].unique())

In [ ]:
# 모든 csv 파일의 상태_B로 변화는 시점이 같다라고 가정
# 하지만, 개별 csv파일의 상태_B로 변화는 시점은 상이할 수 있음
def data_loader_all_v2(func, files, folder='', train_label=None, event_time=10, nrows=60):   
    func_fixed = partial(func, folder=folder, train_label=train_label, event_time=event_time, nrows=nrows)     
    if __name__ == '__main__':
        pool = Pool(processes=multiprocessing.cpu_count()) 
        df_list = list(pool.imap(func_fixed, files)) 
        pool.close()
        pool.join()        
    combined_df = pd.concat(df_list)    
    return combined_df

In [ ]:
train = data_loader_all_v2(data_loader_v2, train_list, folder=train_folder, train_label=train_label, 
                           event_time=10, nrows=120)


In [ ]:
X_train = train.drop(['label'], axis=1)
y_train = train['label']
X_train.shape

In [ ]:
model_ts = datetime.now().strftime('%Y%m%dT%H%M%S')
print('model_ts', model_ts)

initscore_filename = ''
params = {
#     'boosting':'gbdt',
#     'boosting':'dart',
#     'boosting':'goss',
    'num_leaves': 5, 
    'max_depth': 3,
    'num_class':num_class,
    'objective': 'multiclass',
    'metric':'multi_logloss',
    'num_threads': 14,
    'learning_rate': 0.01,
    'is_unbalance': True,
#     'scale_pos_weight':200,
    'bagging_fraction':0.1,
    'bagging_freq':10,
    'feature_fraction':0.1,
    'initscore_filename':initscore_filename,
#     'lambda_l1':200,
#     'lambda_l2':5,
#     'device_type':'gpu',
#     'tree_learner':'data',

}
print(params)

data_params = {
    'max_bin':127,
#     'enable_bundle': False,
}
print(data_params)

num_round = 500
print('num_round:', num_round)

In [ ]:
train_set = lgb.Dataset(X_train, label=y_train, params=data_params)

evals_result = {}
model = lgb.train(params, train_set, num_round, early_stopping_rounds=300, 
                        valid_sets=[train_set],
                        verbose_eval=50,
                          evals_result=evals_result,
                        init_model=model,
                       )

evals_result['training']['multi_logloss'][-1]

In [ ]:
model_tag ='{}_{}'.format(datetime.now().strftime('%Y%m%dT%H%M%S'), evals_result['training']['multi_logloss'][-1])
print(model_tag)

joblib.dump(model, 'model/{}.model'.format(model_tag))

In [ ]:
test = data_loader_all_v2(data_loader_v2, test_list, folder=test_folder, train_label=None, event_time=10, nrows=None)

pred = model.predict(test)

In [ ]:
submission = pd.DataFrame(data=pred)
submission.index = test.index
submission.index.name = 'id'
submission = submission.sort_index()
submission = submission.groupby('id').mean()

submission.to_csv('submit/{}.csv'.format(model_tag), index=True) 
model_tag